<a href="https://colab.research.google.com/github/nurfnick/COVID_Travel/blob/master/Combining_Data_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pa
import numpy as np

In [42]:
COVID = pa.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
travel = pa.read_csv('https://raw.githubusercontent.com/nurfnick/COVID_Travel/master/google_mobility_US_only.csv')
population = pa.read_csv('https://raw.githubusercontent.com/nurfnick/COVID_Travel/master/us_census_with_fips.csv',dtype={"STATE": str,"COUNTY": str})

In [43]:
COVID

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,...,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,154,154,154,154,154,165,165,165,166,166,167,170,171,172,172,172,175,175,177,179,179,179,179,179,180,180,183,183,183,183,185,186,188,192,200,222,222,222,225,226
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,21,21,21,21,21,22,22,22,22,22,22,22,22,22,22,22,22,23,24,26,26,26,27,28,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.220800,-66.590100,"Puerto Rico, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2589,2646,2710,2805,2866,2913,3030,3100,3189,3260,3324,3397,3486,3647,3718,3776,3873,3935,4023,4508,4620,4915,4985,5046,5185,5329,5352,5536,5690,5811,5890,5951,6003,6111,6195,6463,6525,6564,6685,6820
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.335800,-64.896300,"Virgin Islands, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,70,70,70,71,71,71,71,71,71,72,72,72,72,72,72,72,73,73,73,73,76,76,76,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3256,84070016,US,USA,840,NaN,Central Utah,Utah,US,39.372319,-111.575868,"Central Utah, Utah, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,29,30,30,30,31,32,34,33,33,33,33,33,34,38,38,38,38,38,39,39,42,45,46,51,48,50,54,61,70,75,77,79,81,81,89,96,102,110,114,117
3257,84070017,US,USA,840,NaN,Southeast Utah,Utah,US,38.996171,-110.701396,"Southeast Utah, Utah, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,14,14,15,16,16,17,18,18,20,21,21,23,23,23,24,24,24,26,26,26,26,26,26,26,28,28,28,28,29,29,29,29,29,30,31,31,31,31,31,32
3258,84070018,US,USA,840,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,"Southwest Utah, Utah, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,209,219,236,245,260,271,281,294,305,324,330,335,360,386,423,440,454,476,498,524,554,584,616,637,655,680,718,739,780,806,827,855,889,919,974,1031,1056,1104,1151,1200
3259,84070019,US,USA,840,NaN,TriCounty,Utah,US,40.124915,-109.517442,"TriCounty, Utah, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,19,19,22,19,20,20,21,20,20,20,20,20,20,18,20,21,21,20,20,20,20,20,22,22,22,23,27,27,29,30,30,31,33,33,38,38,38,39,39,40


In [44]:
travel

,country_region_code,country_region,sub_region_1,sub_region_2,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,US,United States,NaN,NaN,NaN,NaN,2/15/2020,6.0,2.0,15.0,3.0,2.0,-1.0
1,US,United States,NaN,NaN,NaN,NaN,2/16/2020,7.0,1.0,16.0,2.0,0.0,-1.0
2,US,United States,NaN,NaN,NaN,NaN,2/17/2020,6.0,0.0,28.0,-9.0,-24.0,5.0
3,US,United States,NaN,NaN,NaN,NaN,2/18/2020,0.0,-1.0,6.0,1.0,0.0,1.0
4,US,United States,NaN,NaN,NaN,NaN,2/19/2020,2.0,0.0,8.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
322912,US,United States,Wyoming,Weston County,NaN,56045.0,6/8/2020,NaN,NaN,NaN,NaN,-27.0,NaN
322913,US,United States,Wyoming,Weston County,NaN,56045.0,6/9/2020,NaN,NaN,NaN,NaN,-29.0,NaN
322914,US,United States,Wyoming,Weston County,NaN,56045.0,6/10/2020,NaN,NaN,NaN,NaN,-30.0,NaN
322915,US,United States,Wyoming,Weston County,NaN,56045.0,6/11/2020,NaN,NaN,NaN,NaN,-20.0,NaN


In [45]:
population

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,POPESTIMATE2019
0,40,3,6,01,000,Alabama,Alabama,4779736,4903185
1,50,3,6,01,001,Alabama,Autauga County,54571,55869
2,50,3,6,01,003,Alabama,Baldwin County,182265,223234
3,50,3,6,01,005,Alabama,Barbour County,27457,24686
4,50,3,6,01,007,Alabama,Bibb County,22915,22394
...,...,...,...,...,...,...,...,...,...
3188,50,4,8,56,037,Wyoming,Sweetwater County,43806,42343
3189,50,4,8,56,039,Wyoming,Teton County,21294,23464
3190,50,4,8,56,041,Wyoming,Uinta County,21118,20226
3191,50,4,8,56,043,Wyoming,Washakie County,8533,7805


In [46]:
population["FIPS"] = population["STATE"]+population["COUNTY"] #combining the strings to make the fip value 
population["FIPS"] = population["FIPS"].astype(float)

In [47]:
fips = [x for x in np.unique(travel['census_fips_code']) if ~np.isnan(x)] #get the unique fips and get rid of nans

In [48]:
len(fips)

2832

In [49]:
fips2 = [x for x in np.unique(COVID['FIPS']) if ~np.isnan(x)]

In [50]:
len(fips2) #wtf...

3251

In [51]:
fips3 = [x for x in np.unique(population['FIPS'])]

In [52]:
len(fips3)

3193

So which one is true...  I think I will use the smallest one.  That way I don't have counties missing large chunks of data.

In [75]:
COVID[COVID['FIPS']==1001]
  

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,...,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20
5,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,110,110,120,127,136,143,149,155,159,168,189,192,205,212,216,221,233,239,239,241,248,259,265,272,282,295,312,323,331,357,364,367,369,394,405,425,428,436,447,463


In [30]:
travel[travel['census_fips_code']==1001]

,country_region_code,country_region,sub_region_1,sub_region_2,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
242,US,United States,Alabama,Autauga County,NaN,1001.0,2/15/2020,5.0,7.0,NaN,NaN,-4.0,NaN
243,US,United States,Alabama,Autauga County,NaN,1001.0,2/16/2020,0.0,1.0,-23.0,NaN,-4.0,NaN
244,US,United States,Alabama,Autauga County,NaN,1001.0,2/17/2020,8.0,0.0,NaN,NaN,-27.0,5.0
245,US,United States,Alabama,Autauga County,NaN,1001.0,2/18/2020,-2.0,0.0,NaN,NaN,2.0,0.0
246,US,United States,Alabama,Autauga County,NaN,1001.0,2/19/2020,-2.0,0.0,NaN,NaN,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,US,United States,Alabama,Autauga County,NaN,1001.0,6/10/2020,7.0,11.0,NaN,NaN,-27.0,9.0
359,US,United States,Alabama,Autauga County,NaN,1001.0,6/11/2020,1.0,17.0,NaN,NaN,-25.0,7.0
360,US,United States,Alabama,Autauga County,NaN,1001.0,6/12/2020,-8.0,15.0,NaN,NaN,-25.0,8.0
361,US,United States,Alabama,Autauga County,NaN,1001.0,6/13/2020,-6.0,17.0,NaN,NaN,-3.0,NaN


In [35]:
population[population['FIPS']==1001]

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,POPESTIMATE2019,FIPS
1,50,3,6,01,001,Alabama,Autauga County,54571,55869,1001.0


In [36]:
days = COVID.columns[11:]

In [54]:
travel[travel['date']==days[0]]["workplaces_percent_change_from_baseline"]

Series([], Name: workplaces_percent_change_from_baseline, dtype: float64)

In [93]:
df = pa.DataFrame(columns=['FIPS','County','State','Population','Date','COVID Cases','Workplace Travel'])

In [124]:
i = 0 #initialize

for fip in fips:
  COVIDfip = COVID[COVID["FIPS"]==fip]
  if COVIDfip.size != 0: #make sure it finds that county 
    travelfip = travel[travel['census_fips_code']==fip]
    populationfip = population[population['FIPS']==fip]
    countydata = []
    countydata.append(fip)
    countydata.append(COVIDfip.iloc[0,5]) #county
    countydata.append(COVIDfip.iloc[0,6]) #state value
    countydata.append(populationfip.iloc[0,-2]) #county population estimate 2019
    for day in days: #days are in m/d/yy
      datedata = []
      for item in countydata: #having assignment issues just iterate through values
        datedata.append(item)
      datedata.append(day)
      datedata.append(COVIDfip[day].values[0])
      travelfipday = travelfip[travelfip['date']==day+'20'] #here they are m/d/2020
      if travelfipday.size == 0:
        datedata.append('Nan')
      else:
        datedata.append(travelfipday['workplaces_percent_change_from_baseline'].values[0]) #.values[0] removes the index and column name
      df.loc[i]=datedata
      i+=1 #increment the counter



KeyboardInterrupt: ignored

In [125]:
df.to_csv("Combined.csv")

In [126]:
df

,FIPS,County,State,Population,Date,COVID Cases,Workplace Travel
0,1001.0,Autauga,Alabama,55869,1/22/20,0,Nan
1,1001.0,Autauga,Alabama,55869,1/23/20,0,Nan
2,1001.0,Autauga,Alabama,55869,1/24/20,0,Nan
3,1001.0,Autauga,Alabama,55869,1/25/20,0,Nan
4,1001.0,Autauga,Alabama,55869,1/26/20,0,Nan
...,...,...,...,...,...,...,...
378887,48425.0,Somervell,Texas,9128,3/29/20,0,NaN
378888,48425.0,Somervell,Texas,9128,3/30/20,0,-37
378889,48425.0,Somervell,Texas,9128,3/31/20,0,-32
378890,48425.0,Somervell,Texas,9128,4/1/20,0,-32
